# Get GDELT Schema

In [67]:
import urllib.request  # lib that handles URLs
#import requests
import io
import pandas as pd
pd.options.display.max_rows=250

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data))
colnames_events = list(formats["tableId"])

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data),sep="\t")
colnames_mentions = list(formats.loc[:15,"0"])

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data),sep="\t")
colnames_knowledgegraph = list(formats["tableId"])

In [73]:
len(colnames_knowledgegraph)

27

# Spark Dataset Import

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [147]:
df_events = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/events/20180101*.export.csv")
df_events = df_events.toDF(*colnames_events)
df_events.cache()

DataFrame[GLOBALEVENTID: int, SQLDATE: int, MonthYear: int, Year: int, FractionDate: double, Actor1Code: string, Actor1Name: string, Actor1CountryCode: string, Actor1KnownGroupCode: string, Actor1EthnicCode: string, Actor1Religion1Code: string, Actor1Religion2Code: string, Actor1Type1Code: string, Actor1Type2Code: string, Actor1Type3Code: string, Actor2Code: string, Actor2Name: string, Actor2CountryCode: string, Actor2KnownGroupCode: string, Actor2EthnicCode: string, Actor2Religion1Code: string, Actor2Religion2Code: string, Actor2Type1Code: string, Actor2Type2Code: string, Actor2Type3Code: string, IsRootEvent: int, EventCode: int, EventBaseCode: int, EventRootCode: int, QuadClass: int, GoldsteinScale: double, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: double, Actor1Geo_Type: int, Actor1Geo_FullName: string, Actor1Geo_CountryCode: string, Actor1Geo_ADM1Code: string, Actor1Geo_ADM2Code: string, Actor1Geo_Lat: double, Actor1Geo_Long: double, Actor1Geo_FeatureID: string,

In [86]:
df_mentions = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/mentions/20180101*.mentions.csv")
df_mentions = df_mentions.toDF(*colnames_mentions)
df_mentions.cache()

DataFrame[GLOBALEVENTID: int, EventTimeDate: bigint, MentionTimeDate: bigint, MentionType: int, MentionSourceName: string, MentionIdentifier: string, SentenceID: int, Actor1CharOffset: int, Actor2CharOffset: int, ActionCharOffset: int, InRawText: int, Confidence: int, MentionDocLen: int, MentionDocTone: double, MentionDocTranslationInfo: string, Extras: string]

In [142]:
df_gkg = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load("s3://gdelt-open-data/v2/gkg/2018010*.gkg.csv")
df_gkg = df_gkg.toDF(*colnames_knowledgegraph)
df_gkg.cache()

DataFrame[GKGRECORDID: string, DATE: bigint, SourceCollectionIdentifier: int, SourceCommonName: string, DocumentIdentifier: string, Counts: string, V2Counts: string, Themes: string, V2Themes: string, Locations: string, V2Locations: string, Persons: string, V2Persons: string, Organizations: string, V2Organizations: string, V2Tone: string, Dates: string, GCAM: string, SharingImage: string, RelatedImages: string, SocialImageEmbeds: string, SocialVideoEmbeds: string, Quotations: string, AllNames: string, Amounts: string, TranslationInfo: string, Extras: string]

In [143]:
df_gkg.count()

1859033

In [103]:
df_gkg.printSchema()

root
 |-- GKGRECORDID: string (nullable = true)
 |-- DATE: long (nullable = true)
 |-- SourceCollectionIdentifier: integer (nullable = true)
 |-- SourceCommonName: string (nullable = true)
 |-- DocumentIdentifier: string (nullable = true)
 |-- Counts: string (nullable = true)
 |-- V2Counts: string (nullable = true)
 |-- Themes: string (nullable = true)
 |-- V2Themes: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- V2Locations: string (nullable = true)
 |-- Persons: string (nullable = true)
 |-- V2Persons: string (nullable = true)
 |-- Organizations: string (nullable = true)
 |-- V2Organizations: string (nullable = true)
 |-- V2Tone: string (nullable = true)
 |-- Dates: string (nullable = true)
 |-- GCAM: string (nullable = true)
 |-- SharingImage: string (nullable = true)
 |-- RelatedImages: string (nullable = true)
 |-- SocialImageEmbeds: string (nullable = true)
 |-- SocialVideoEmbeds: string (nullable = true)
 |-- Quotations: string (nullable = true)
 |-- AllN

# Basic Queries

In [152]:
print(recs["SOURCEURL"][0])

http://www.kansas.com/news/business/article192357844.html


In [149]:
test = df_events.take(1)
recs = pd.DataFrame(test)
recs.columns = colnames_events
print(recs.T)

                                                                       0
GLOBALEVENTID                                                  719024869
SQLDATE                                                         20170101
MonthYear                                                         201701
Year                                                                2017
FractionDate                                                        2017
Actor1Code                                                           AGR
Actor1Name                                                        FARMER
Actor1CountryCode                                                   None
Actor1KnownGroupCode                                                None
Actor1EthnicCode                                                    None
Actor1Religion1Code                                                 None
Actor1Religion2Code                                                 None
Actor1Type1Code                                    

In [170]:
test = df_events.where((df_events.Actor1Name.rlike('PRESIDENT'))&(df_events.Actor1Code=='GOV')).collect()
recs = pd.DataFrame(test)
recs.columns = colnames_events
recs.head().T

,0,1,2,3,4
GLOBALEVENTID,719025693,719025698,719025699,719025700,719025708
SQLDATE,20180101,20180101,20180101,20180101,20180101
MonthYear,201801,201801,201801,201801,201801
Year,2018,2018,2018,2018,2018
FractionDate,2018,2018,2018,2018,2018
Actor1Code,GOV,GOV,GOV,GOV,GOV
Actor1Name,PRESIDENT,PRESIDENT,PRESIDENT,PRESIDENT,PRESIDENT
Actor1CountryCode,None,None,None,None,None
Actor1KnownGroupCode,None,None,None,None,None
Actor1EthnicCode,None,None,None,None,None


In [178]:
#719025693
#test = df_mentions.where(df_mentions.MentionIdentifier.rlike('tariff')).take(5)
test = df_mentions.where(df_mentions.GLOBALEVENTID == 719025693).collect()
recs = pd.DataFrame(test)
recs.columns = colnames_mentions
recs

,GLOBALEVENTID,EventTimeDate,MentionTimeDate,MentionType,MentionSourceName,MentionIdentifier,SentenceID,Actor1CharOffset,Actor2CharOffset,ActionCharOffset,InRawText,Confidence,MentionDocLen,MentionDocTone,MentionDocTranslationInfo,Extras
0,719025693,20180101000000,20180101171500,1,theadvertiser.com,http://www.theadvertiser.com/story/opinion/col...,2,930,-1,989,1,50,5262,-4.969697,None,None
1,719025693,20180101000000,20180101203000,1,therecord.com,https://www.therecord.com/news-story/8031262-i...,7,-1,1724,1749,1,40,5503,-0.265252,None,None
2,719025693,20180101000000,20180101160000,1,dailyworld.com,http://www.dailyworld.com/story/opinion/column...,2,895,-1,954,1,50,5140,-5.006105,None,None
3,719025693,20180101000000,20180101161500,1,tallahassee.com,http://www.tallahassee.com/story/opinion/2018/...,3,881,-1,940,1,80,5417,-4.389087,None,None
4,719025693,20180101000000,20180101194500,1,niagarathisweek.com,https://www.niagarathisweek.com/news-story/803...,7,-1,1724,1749,1,40,5576,-0.259740,None,None
5,719025693,20180101000000,20180101213000,1,durhamregion.com,https://www.durhamregion.com/news-story/803126...,7,-1,1724,1749,1,40,5743,-0.889454,None,None
6,719025693,20180101000000,20180101213000,1,hamiltonnews.com,https://www.hamiltonnews.com/news-story/803126...,7,-1,1724,1749,1,40,5689,-0.636132,None,None
7,719025693,20180101000000,20180101213000,1,scmp.com,http://www.scmp.com/news/world/united-states-c...,12,-1,2977,3002,1,60,5188,-2.038505,None,None
8,719025693,20180101000000,20180101011500,1,newsweek.com,http://www.newsweek.com/trump-happy-new-year-f...,3,2002,-1,2141,1,30,2120,-4.054054,None,None
9,719025693,20180101000000,20180101224500,1,sfgate.com,http://www.sfgate.com/world/article/Israeli-ru...,7,-1,1343,1368,1,40,2829,-0.431965,None,None


In [145]:
test = df_gkg.where(df_gkg.V2Themes.rlike('TARIFF')).take(1)
cols = colnames_knowledgegraph

recs = pd.DataFrame(test)
recs.columns = cols
print(recs.T)

                                                                            0
GKGRECORDID                                                20180109194500-602
DATE                                                           20180109194500
SourceCollectionIdentifier                                                  1
SourceCommonName                                        voiceofalexandria.com
DocumentIdentifier          http://www.voiceofalexandria.com/news/national...
Counts                                                                   None
V2Counts                                                                 None
Themes                      ECON_STOCKMARKET;GENERAL_HEALTH;MEDICAL;EPU_CA...
V2Themes                    TAX_FNCACT_TROOPERS,5220;CRISISLEX_CRISISLEXRE...
Locations                   1#Germany#GM#GM#51.5#10.5#GM;1#Japan#JA#JA#36#...
V2Locations                 3#Boston, Massachusetts, United States#US#USMA...
Persons                     donald trump;terrence byrd;goldman s

In [146]:
df_gkg.where((df_gkg.V2Persons.rlike('Donald Trump')) &
             (df_gkg.V2Themes.rlike('TARIFF') | df_gkg.V2Themes.rlike('TRADE'))
).count()

13632

In [141]:
df_gkg.select("V2Themes").distinct().count()

74622

# Stop Spark Context

In [ ]:
sc.stop()